In [51]:
# imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from Pipeline import preproc
import os
import re

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [52]:
# Load lookup and aligned table for data extraction
aligned = pd.read_excel('COVID_Qualtrics_items_scores_aligned.xlsx', 'items')
lookup = pd.read_excel('COVID_Qualtrics_items_scores_aligned.xlsx', 'items_lookup')

In [53]:
# define the output datafile:
out = {'Week1': 'output/cleaned_data/Week1.csv',
       'Week2': 'output/cleaned_data/Week2.csv',
       'Week3': 'output/cleaned_data/Week3.csv',
       'Week4': 'output/cleaned_data/Week4.csv',
       'Week5': 'output/cleaned_data/Week5.csv',
       'Week6': 'output/cleaned_data/Week6.csv',
       'Wave7': 'output/cleaned_data/Wave7.csv'}

# define the output datafile:
out_tmp = {'Week1': 'output/cleaned_data/tmp/task/Week1.csv',
       'Week2': 'output/cleaned_data/tmp/task/Week2.csv',
       'Week3': 'output/cleaned_data/tmp/task/Week3.csv',
       'Week4': 'output/cleaned_data/tmp/task/Week4.csv',
       'Week5': 'output/cleaned_data/tmp/task/Week5.csv',
       'Week6': 'output/cleaned_data/tmp/task/Week6.csv',
       'Wave7': 'output/cleaned_data/tmp/task/Wave7.csv'}

In [54]:
# define the original data paths
data_files = {'Week1':'Working_Data/Week1/W1_Qualtrics/COVID19_first_PROLIFIC_WEEK1_text_items.csv',
              'Pilot30':'Working_Data/Pilot30/COVID19_first_PROLIFIC_PILOT30_text_items.csv',
              'Week2':'Working_Data/Week2/W2_Qualtrics/COVID19_first_PROLIFIC_WEEK2_text_items.csv',
              'Week3':'Working_Data/Week3/W3_Qualtrics/COVID19_first_PROLIFIC_WEEK3_text.csv',
              'Week4':'Working_Data/Week4/W4_Qualtrics/COVID19_first_PROLIFIC_WEEK4_text_items.csv',
              'Week5':'Working_Data/Week5/W5_Qualtrics/COVID19_first_PROLIFIC_WAVE5_text.csv',
              'Week6':'Working_Data/Wave6/W6_Qualtrics/COVID19_first_PROLIFIC_WAVE6_text.csv',
              'Wave7':'Working_Data/Wave7/W7_Qualtrics/COVID19_first_PROLIFIC_WAVE7_text.csv',
              'Wave8':'Working_Data/Wave8/W8_Qualtrics/COVID19_first_PROLIFIC_WAVE8_text.csv'}
score_files = {'Week1':'Working_Data/Week1/W1_Qualtrics/covid19_first_prolific_week1_scores.csv',
               'Pilot30':'Working_Data/Pilot30/COVID19_first_PROLIFIC_PILOT30_scores.csv',
               'Week2':'Working_Data/Week2/W2_Qualtrics/covid19_first_prolific_week2_scores.csv',
               'Week3':'Working_Data/Week3/W3_Qualtrics/COVID19_first_PROLIFIC_WEEK3_scores.csv',
               'Week4':'Working_Data/Week4/W4_Qualtrics/COVID19_first_PROLIFIC_WEEK4_scores.csv'}
num_files = {'Week1':'Working_Data/Week1/W1_Qualtrics/COVID19_first_PROLIFIC_WEEK1_numbers_items.csv',
              'Pilot30':'Working_Data/Pilot30/COVID19_first_PROLIFIC_PILOT30_number_items.csv',
              'Week2':'Working_Data/Week2/W2_Qualtrics/COVID19_first_PROLIFIC_WEEK2_num_items.csv',
              'Week3':'Working_Data/Week3/W3_Qualtrics/COVID19_first_PROLIFIC_WEEK3_num.csv',
              'Week4':'Working_Data/Week4/W4_Qualtrics/COVID19_first_PROLIFIC_WEEK4_num_items.csv',
              'Week5':'Working_Data/Week5/W5_Qualtrics/COVID19_first_PROLIFIC_WAVE5_num.csv',
              'Week6':'Working_Data/Wave6/W6_Qualtrics/COVID19_first_PROLIFIC_WAVE6_num.csv',
              'Wave7':'Working_Data/Wave7/W7_Qualtrics/COVID19_first_PROLIFIC_WAVE7_num.csv',
              'Wave8':'Working_Data/Wave8/W8_Qualtrics/COVID19_first_PROLIFIC_WAVE8_num.csv'}
randomizer_files = {'Week1': 'Working_Data/Week1/W1_Qualtrics/COVID19_first_PROLIFIC_WEEK1_Randomizer.csv',
                    'Pilot30':'Working_Data/Pilot30/COVID19_first_PROLIFIC_PILOT30 _Randomizer.csv',
                    'Week2':'Working_Data/Week2/W2_Qualtrics/COVID19_first_PROLIFIC_WAVE2_Randomizer.csv',
                    'Week3':'Working_Data/Week3/W3_Qualtrics/COVID19_first_PROLIFIC_WAVE3_Randomizer.csv',
                    'Week4':'Working_Data/Week4/W4_Qualtrics/COVID19_first_PROLIFIC_WAVE4_Randomizer.csv',
                    'Week5':'Working_Data/Week5/W5_Qualtrics/COVID19_first_PROLIFIC_WAVE5_Randomizer.csv',
                    'Week6':'Working_Data/Wave6/W6_Qualtrics/COVID19_first_PROLIFIC_WAVE6_Randomizer.csv',
                    'Wave7':'Working_Data/Wave7/W7_Qualtrics/COVID19_first_PROLIFIC_WAVE7_Randomizer.csv',
                    'Wave8':'Working_Data/Wave8/W8_Qualtrics/COVID19_first_PROLIFIC_WAVE8_Randomizer.csv'}

In [55]:
# define useful variables
save_path = 'output/cleaned_data/tmp/'

In [56]:
def clean_column_names(col_names):
    # Clean up the column names by removing the unicode values
    new_cols = []
    for col in col_names:
        replaced = re.sub(r'[^\x00-\x7F]', '', col)
        new_cols.append(replaced)
    return new_cols


def reformat_conte_CCID(data):
    tmp = []
    for CCID in data:
        if len(CCID) == 1:
            tmp.append('CC000' + CCID)
        elif len(CCID) == 2:
            tmp.append('CC00' + CCID)
        elif len(CCID) == 3:
            tmp.append('CC0' + CCID)
        elif len(CCID) == 4:
            tmp.append('CC' + CCID)
        else:
            tmp.append(CCID)
    return tmp


def detect_new_columns(data, aligned, week):
    cols = []
    for col in list(data.columns):
        if col not in list(aligned[week]):
            cols.append(col)
    return cols


def ext_txt(data, week, aligned, lookup, debug=False, debug_col=None):

    # Column types to ignore:
    ig_cols = ['unchanged', 'text', 'numeric']

    # Preprocessing for the lookup table
    # Get rid of all unicode characters and punctuation characters using regular expression
    # Change all response values in the lookup table to lowercase letters
    lookup.loc[:, 'Responses'] = lookup['Responses'].astype(str).str.lower()\
                                 .str.replace(r"[^\x00-\x7F]", '')\
                                 .str.replace(r"[ .,!\"']", '')

    # Start value substitution
    data_cols = list(data.columns)

    # Clean column names
    data_cols = clean_column_names(data_cols)
    #print(data_cols)
    data.columns = data_cols

    iteration = aligned.iterrows()

    # Skip the first row from the align table
    next(iteration)
    
    # Define a rename dictionary
    tmp_dict = {}

    for row in iteration:
        # Get column names and lookup id from the aligned/items table
        column = row[1][week]
        lookup_id = row[1]['Question_type']
        new_col_name = row[1]['unified_variable_names']

        # Only run for columns that exist in the dataset
        if (column is not np.nan):
            str_col = str(column)
            if str_col in data_cols:
                # If this variable only has a single value, convert it to 1
                if lookup_id == 'single_item':
                    #print('single_item: '+ str_col)
#                     if (str_col=='Mask1_1'):
#                         print(str_col)
#                         print(data[str_col])
#                         print(new_col_name)
                    idx = data[str_col].notna()
                    idx[0] = False
                    data.loc[idx, str_col] = 1.0
                    
                    

                # If other response type, create a substitution dictionary
                elif lookup_id not in ig_cols:
                    if str_col == 'GFPS7_1':
                        print(str_col)
                    # Generate the dictionary for replacement
                    replace_dict = lookup.loc[lookup['Type']==lookup_id, ['Responses', 'Numeric']]\
                                   .set_index('Responses').to_dict()
                    
                    if str_col == 'GFPS7_1':
                        print(replace_dict)
                        print(data[str_col].unique())
                        
                    data.loc[1:,str_col] = data.loc[1:,str_col].str.lower().\
                                           str.replace(r"[^\x00-\x7F]", '').\
                                           str.replace(r"[ .,!\"']", '').\
                                           replace(replace_dict['Numeric']).astype(float)
                
                tmp_dict[str_col] = new_col_name
#                 if str_col == 'Mask1_1':
#                     print(new_col_name)
#                     print(data['Mask1_1'])
                    

            # A warning for situation that a column is found in the aligned table but not in the input data
            else:
                print("Warning: column '{}' in aligned file is not a column of the data file.".format(str_col))
        
    data = data.rename(columns=tmp_dict)
        
    return data

def handle_RW27(num, data, week):
    data.columns = clean_column_names(list(data.columns))
    num.columns = clean_column_names(list(num.columns))
    print(num.columns)
    if (week == 'Pilot30') | (week == 'Week1'):
        col = 'Q60'
    else:
        col = 'RW27'
    
    iteration = data.iterrows()
    next(iteration)
    for index, row in iteration:
        rid = row['V1']
        val = num.loc[num['V1']==rid, col]
        if (~val.isna().values):
            data.loc[data['V1']==rid, col] = int(num.loc[num['V1']==rid, col])
    
    return data

def comb_score(score_file, processed_data, week):
    score = pd.read_csv(score_file, encoding = "ISO-8859-1", dtype=str)
    score_cols = list(score.columns)
    # Clean column names
    score_cols = clean_column_names(score_cols)
    score.columns = score_cols
    
    #print(score.loc[score['PROLIFIC_PID']=='5def020419398a5b744b98b9', "SNI_Ex_Network_Diversity_allComm_2019"])
    
    if week.endswith('C'):
        # handle conte data
        score['CCID'] = reformat_conte_CCID(score['CCID'])
        processed_data = processed_data.merge(score, how='left', on='CCID')
    else:
        processed_data = processed_data.merge(score, how='left', on='PROLIFIC_PID')
    #print(processed_data.loc[processed_data['PROLIFIC_PID']=='5def020419398a5b744b98b9', "SNI_Ex_Network_Diversity_allComm_2019"])
    return processed_data


def extract(source, num, score, week_col, aligned, lookup, save_path=None):
    # Somehow the new files we work with requires a different encoding.
    data = pd.read_csv(source, encoding = "ISO-8859-1", dtype=str, keep_default_na=False, na_values=['', 'nan'])
    data['PROLIFIC_PID'] = data['PROLIFIC_PID'].str.replace(' ', '')

    num_data = pd.read_csv(num, encoding = "ISO-8859-1", dtype=str, keep_default_na=False, na_values=['', 'nan'])
    # First handle the RW27 and Q60 column
    num_data['PROLIFIC_PID'] = num_data['PROLIFIC_PID'].str.replace(' ', '')

    data = handle_RW27(num_data, data, week=week_col)
    processed_data = ext_txt(data, week_col, aligned, lookup)
    processed_data = comb_score(score, processed_data, week_col)

    if save_path:
        if not os.path.exists(save_path):
            os.makedirs(save_path)

        path = save_path + week_col + '.csv'
        processed_data.to_csv(path, index=False, encoding='utf-8')

    return processed_data

def clean_column_names(col_names):
    # Clean up the column names by removing the unicode values
    new_cols = []
    for col in col_names:
        replaced = re.sub(r'[^\x00-\x7F]', '', col)
        new_cols.append(replaced)
    return new_cols


def unify_randomizer_names(data, week, aligned, lookup, debug=False, debug_col=None):
    # Column types to ignore:
    ig_cols = ['unchanged', 'text', 'numeric']

    # Preprocessing for the lookup table
    # Get rid of all unicode characters and punctuation characters using regular expression
    # Change all response values in the lookup table to lowercase letters
    lookup.loc[:, 'Responses'] = lookup['Responses'].astype(str).str.lower()\
                                 .str.replace(r"[^\x00-\x7F]", '')\
                                 .str.replace(r"[ .,!\"']", '')

    # Start value substitution
    data_cols = list(data.columns)

    # Clean column names
    data_cols = clean_column_names(data_cols)
    #print(data_cols)
    data.columns = data_cols

    iteration = aligned.iterrows()

    # Skip the first row from the align table
    next(iteration)
    
    # Define a rename dictionary
    tmp_dict = {}

    for row in iteration:
        # Get column names and lookup id from the aligned/items table
        column = row[1][week]
        lookup_id = row[1]['Question_type']
        new_col_name = row[1]['unified_variable_names']

        # Only run for columns that exist in the dataset
        if (column is not np.nan):
            str_col = str(column)
            if str_col in data_cols:
                tmp_dict[str_col] = new_col_name
#                 if str_col == 'Mask1_1':
#                     print(new_col_name)
#                     print(data['Mask1_1'])
                
            # A warning for situation that a column is found in the aligned table but not in the input data
            else:
                print("Warning: column '{}' in aligned file is not a column of the data file.".format(str_col))
        
    data = data.rename(columns=tmp_dict)
        
    return data


### Process regular data and extract from the text files

In [57]:
# Processing for Week1 + Pilot
pilot_data = extract(data_files['Pilot30'], num_files['Pilot30'], score_files['Pilot30'], 'Pilot30', aligned, lookup, save_path)
week1_data = extract(data_files['Week1'], num_files['Week1'], score_files['Week1'], 'Week1', aligned, lookup, save_path)
w1_randomizer = pd.read_csv(randomizer_files['Week1'], encoding = "ISO-8859-1", dtype=str, keep_default_na=False, na_values=['', 'nan'])


Index(['V1', 'V2', 'V3', 'V4', 'V5', 'PROLIFIC_PID', 'TASK1', 'TASK2', 'TASK3',
       'TASK4',
       ...
       'Q130_7', 'Q130_7_TEXT', 'Q179', 'Q202', 'Q203', 'Q204', 'Q205', 'Q207',
       'Q364', 'Q239'],
      dtype='object', length=650)
Index(['V1', 'V2', 'V3', 'V4', 'V5', 'PROLIFIC_PID', 'TASK1', 'TASK2', 'TASK3',
       'TASK4',
       ...
       'Q130_7', 'Q130_7_TEXT', 'Q179', 'Q202', 'Q203', 'Q204', 'Q205', 'Q207',
       'Q364', 'Q239'],
      dtype='object', length=650)


In [58]:
# Processing for Week2
week2_data = extract(data_files['Week2'], num_files['Week2'], score_files['Week2'], 'Week2', aligned, lookup, save_path)
w2_randomizer = pd.read_csv(randomizer_files['Week2'], encoding = "ISO-8859-1", dtype=str, keep_default_na=False, na_values=['', 'nan'])
w2_randomizer.columns = clean_column_names(w2_randomizer.columns)
w2_randomizer.drop(['V3', 'V4', 'V5', 'PROLIFIC_PID'], inplace=True, axis=1)
w2_randomizer = unify_randomizer_names(w2_randomizer, 'Week2', aligned, lookup)
week2_data = week2_data.merge(w2_randomizer, how='left', on='V1')
week2_data = preproc.clean_text_columns(week2_data, aligned)
week2_data.to_csv('output/cleaned_data/tmp/Week2.csv', index=False)

Index(['V1', 'V2', 'V3', 'V4', 'V5', 'PROLIFIC_PID', 'CVD', 'PEN', 'TASK1',
       'TASK2',
       ...
       'VSA1_4', 'VSA1_5', 'VSA1_6', 'norm1', 'norm2', 'norm3', 'norm4',
       'longitudinal2', 'feedback', 'Q239'],
      dtype='object', length=770)


V1
V2
V3
V4
V5
PROLIFIC_PID
CVD
PEN
TASK1
TASK2
TASK3
TASK4
TASK5
pavLink
Mobile
PID Missing
browser_1_TEXT
browser_2_TEXT
browser_3_TEXT
browser_4_TEXT
browser_5_TEXT
browser_6_TEXT
browser_7_TEXT
Q4_1
Q4_2
Q4_3
Q4_4
consent
Declined
longitudinal1
commit
longitudinal2
DISG1.1_23
RW23
DemW1
DemW2
DemW3
DemW4
DemW5
DemW6
DemW7
DemW8
DemW9_1
DemW9_2
DemW9_3
DemW9_4
DemW9_5
DemW9_6
DemW9_7
DemW9_8
DemW9_9
DemW9_10
DemW9_11
DemW9_11_TEXT
DemW10_R3
DemW11
DemW12
DemW13
DemW14
DemW15_1
DemW15_2
DemW15_3
DemW15_4
DemW15_5
DemW15_5_TEXT
DemW16
DemW17_1
DemW17_2
DemW17_3
DemW17_3_TEXT
DemW18_R2
DemW19_1
DemW19_2
DemW19_3
DemW19_4
DemW19_5
DemW19_5_TEXT
DemW19_6
DemW20_1
DemW20_2
DemW20_3
DemW20_3_TEXT
DemW20_4
DemW21_1_TEXT
DemW21_2_TEXT
DemW21_3_TEXT
RW1_1
RW1_2
RW1_3
RW1_4
RW1_5
RW1_5_TEXT
RW1_6
RW1_7
RW1_7_TEXT
RW1_8
RW2
RW3
RW4
RW5_1
RW5_2
RW5_3
RW5_4
RW5_5
RW5_6
RW5_7
RW5_8
RW5_9
RW5_10
RW5_11
RW5_12
RW5_13
RW5_14
RW5_15
RW5_16
RW5_16_TEXT
RW5_17
RW5_17_TEXT
RW5_18
RW5_18_TEXT
RW6
RW7_1_TE

In [59]:
# Processing for Week3
week3_data = extract(data_files['Week3'], num_files['Week3'], score_files['Week3'], 'Week3', aligned, lookup, save_path)
w3_randomizer = pd.read_csv(randomizer_files['Week3'], encoding = "ISO-8859-1", dtype=str, keep_default_na=False, na_values=['', 'nan'])
w3_randomizer.columns = clean_column_names(w3_randomizer.columns)
w3_randomizer.drop(['V3', 'V4', 'V5', 'PROLIFIC_PID'], inplace=True, axis=1)
w3_randomizer = unify_randomizer_names(w3_randomizer, 'Week3', aligned, lookup)
week3_data = week3_data.merge(w3_randomizer, how='left', on='V1')
week3_data = preproc.clean_text_columns(week3_data, aligned)
week3_data.to_csv('output/cleaned_data/tmp/Week3.csv', index=False)

Index(['V1', 'V2', 'V3', 'V4', 'V5', 'PROLIFIC_PID', 'CVDcomp', 'PENcomp',
       'CVD', 'PEN',
       ...
       'Theo4_TEXT', 'Theo5', 'Theo5_TEXT', 'Theo6', 'Theo7_1_TEXT',
       'Theo7_2_TEXT', 'longitudinal2', 'feedback', 'resource1', 'resource2'],
      dtype='object', length=964)


V1
V2
V3
V4
V5
PROLIFIC_PID
CVDcomp
PENcomp
CVD
PEN
TASK1
TASK2
TASK3
TASK4
TASK5
pavLink
Mobile
PID Missing
browser_1_TEXT
browser_2_TEXT
browser_3_TEXT
browser_4_TEXT
browser_5_TEXT
browser_6_TEXT
browser_7_TEXT
Q4_1
Q4_2
Q4_3
Q4_4
Q1_1
Q1_2
Q1_3
Q1_4
consent
Declined
longitudinal1
commit
resource_pre
longitudinal2
NIHE1_9
ReSe1_23
RW23
DemW1
DemW2
DemW3
DemW4
DemW5
DemW6
DemW7
DemW8
DemW9_1
DemW9_2
DemW9_3
DemW9_4
DemW9_5
DemW9_6
DemW9_7
DemW9_8
DemW9_9
DemW9_10
DemW9_11
DemW9_11_TEXT
DemW10_R3
DemW11
DemW12
DemW13
DemW14
DemW15_1
DemW15_2
DemW15_3
DemW15_4
DemW15_5
DemW15_5_TEXT
DemW16
DemW17_1
DemW17_2
DemW17_3
DemW17_3_TEXT
DemW18_R2
DemW19_1
DemW19_2
DemW19_3
DemW19_4
DemW19_5
DemW19_5_TEXT
DemW19_6
DemW20_1
DemW20_2
DemW20_3
DemW20_3_TEXT
DemW20_4
DemW21_1_TEXT
DemW21_2_TEXT
DemW21_3_TEXT
DemM9_1
DemM9_2
DemM9_3
DemM9_4
DemM9_5
DemM9_6
DemM9_7
DemM9_8
DemM10_1
DemM10_2
DemM10_3
DemM10_4
DemM10_5
DemM10_6
DemM10_7
DemM10_8
DemM10_9
DemM10_9_TEXT
RW1_1
RW1_2
RW1_3
RW1_4
RW1_5
RW1

DO_Main_Surveys
DO-Q-City
DO-Q-DISG2.1
DO-Q-DISG2.2
DO-Q-EMSB1
DO-Q-EMSC1
DO-Q-Fear2
DO-Q-Fed
DO-Q-RW6_1
DO-Q-State
DO-Q-Thrm1


In [60]:
# Processing for week4
week4_data = extract(data_files['Week4'], num_files['Week4'], score_files['Week4'], 'Week4', aligned, lookup, save_path)
w4_randomizer = pd.read_csv(randomizer_files['Week4'], encoding = "ISO-8859-1", dtype=str, keep_default_na=False, na_values=['', 'nan'])
w4_randomizer.columns = clean_column_names(w4_randomizer.columns)
w4_randomizer.drop(['V3', 'V4', 'V5', 'PROLIFIC_PID'], inplace=True, axis=1)
w4_randomizer = unify_randomizer_names(w4_randomizer, 'Week4', aligned, lookup)

week4_data = week4_data.merge(w4_randomizer, how='left', on='V1')
week4_data = preproc.clean_text_columns(week4_data, aligned)
week4_data.to_csv('output/cleaned_data/tmp/Week4.csv', index=False)

Index(['V1', 'V2', 'V3', 'V4', 'V5', 'PROLIFIC_PID', 'CVDcomp', 'PENcomp',
       'CVD', 'PEN',
       ...
       'VSA1_5', 'VSA1_6', 'norm1', 'norm2', 'norm3', 'norm4', 'longitudinal2',
       'feedback', 'resource1', 'resource2'],
      dtype='object', length=649)


V1
V2
V3
V4
V5
PROLIFIC_PID
CVDcomp
PENcomp
CVD
PEN
TASK1
TASK2
TASK3
TASK4
TASK5
pavLink
Mobile
PID Missing
browser_1_TEXT
browser_2_TEXT
browser_3_TEXT
browser_4_TEXT
browser_5_TEXT
browser_6_TEXT
browser_7_TEXT
Q4_1
Q4_2
Q4_3
Q4_4
Q1_1
Q1_2
Q1_3
Q1_4
consent
Declined
longitudinal1
commit
resource_pre
longitudinal2
DISG2.2_23
Fed_13
EES1_32
RW23
RP1
RP2
RP3
RP4
RP5
RP6
RP7
RP8
RP9_1
RP9_2
RP9_3
RP9_4
RP9_5
RP9_5_TEXT
RP10
DemW1
DemW2
DemW3
DemW4
DemW5
DemW6
DemW7
DemW8
DemW9_1
DemW9_2
DemW9_3
DemW9_4
DemW9_5
DemW9_6
DemW9_7
DemW9_8
DemW9_9
DemW9_10
DemW9_11
DemW9_11_TEXT
DemW10_R3
DemW11
DemW12
DemW13
DemW14
DemW15_1
DemW15_2
DemW15_3
DemW15_4
DemW15_5
DemW15_5_TEXT
DemW16
DemW17_1
DemW17_2
DemW17_3
DemW17_3_TEXT
DemW18_R2
DemW19_1
DemW19_2
DemW19_3
DemW19_4
DemW19_5
DemW19_5_TEXT
DemW19_6
DemW20_1
DemW20_2
DemW20_3
DemW20_3_TEXT
DemW20_4
DemW21_1_TEXT
DemW21_2_TEXT
DemW21_3_TEXT
DemM1_1
DemM1_2
DemM1_3
DemM1_4
DemM1_5
DemM1_6
DemM1_7
DemM2
DemM3
DemM4
DemM5
DemM6
DemM6_TEXT
DemM7
De

#### Combine Week1 and Pilot

In [61]:
# Rename week1 and pilot data
week1_data = pd.read_csv('output/cleaned_data/tmp/Week1.csv', dtype=str,keep_default_na=False, na_values=['', 'nan'])
pilot_data = pd.read_csv('output/cleaned_data/tmp/Pilot30.csv', dtype=str,keep_default_na=False, na_values=['', 'nan'])

week1_data = week1_data.rename(columns={'Disgust_raw_total':'Disgust_total_raw',
                               'NEO_N_zscore': 'NEO_N_z-score',
                               'NEO_E_zscore': 'NEO_E_z-score',
                               'NEO_O_zscore': 'NEO_O_z-score',
                               'NEO_A_zscore': 'NEO_A_z-score',
                               'NEO_C_zscore': 'NEO_C_z-score'})
pilot_data = pilot_data.drop(['SNI_Ex_Non_Relative_Typical_2019'], axis=1)
week1_data = pd.concat([week1_data, pilot_data.iloc[1:]], axis=0)

/Users/Work/opt/anaconda3/envs/g/lib/python3.6/site-packages/ipykernel_launcher.py:12: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  if sys.path[0] == '':


In [62]:
w1_randomizer.columns = clean_column_names(w1_randomizer.columns)
w1_randomizer.drop(['V2', 'V3', 'V4', 'V5', 'PROLIFIC_PID'], inplace=True, axis=1)
w1_randomizer = unify_randomizer_names(w1_randomizer, 'Week1', aligned, lookup)

week1_data = week1_data.merge(w1_randomizer, how='left', on='V1')
week1_data = preproc.clean_text_columns(week1_data, aligned)
week1_data.to_csv('output/cleaned_data/tmp/Week1.csv', index=False)

V1
V2
V3
V4
V5
PROLIFIC_PID
TASK1
TASK2
TASK3
TASK4
pavLink
Mobile
PID Missing
browser_1_TEXT
browser_2_TEXT
browser_3_TEXT
browser_4_TEXT
browser_5_TEXT
browser_6_TEXT
browser_7_TEXT
Q4_1
Q4_2
Q4_3
Q4_4
consent
Declined
longitudinal1
commit
longitudinal2
DemC25
DISG1.1_23
RW23
 
DemW1
DemW3
DemW5
DemW10_R1
DemW11
DemW12
DemW13
DemW14
DemW15_1
DemW15_2
DemW15_3
DemW15_4
DemW15_5
DemW16
DemW17_1
DemW17_2
DemW17_3
DemW17_3_TEXT
DemW18_R1
DemC5
DemC6_1
DemC6_2
DemC6_3
DemC6_4
DemC6_5
DemC6_6
DemC6_7
DemC6_8
DemC6_9
DemC6_10
DemC6_10_TEXT
DemC6_11
DemC7_1
DemC7_2


DemC7_3
DemC7_4
DemC7_5
DemC7_6
DemC7_7
DemC7_8
DemC7_9
DemC7_10
DemC7_10_TEXT
DemC7_11
DemC8
DemC9
DemC9_TEXT
DemM1_1
DemM1_2
DemM1_3
DemM1_4
DemM1_5
DemM1_6
DemM1_7
DemM2
DemC12
DemC13
DemM3
DemM4
DemM5
DemM6
DemM6_TEXT
DemM7
DemC19_1
DemC19_2
DemC19_3
DemC19_4
DemC19_5
DemC19_8
DemC19_9
DemC19_10
Q225
DemC21
DemC22
DemC23
DemC23_TEXT
DemC26_1
DemC26_2
DemC26_3
DemC26_4
DemC26_5
DemC26_6
DemC26_6_TEXT
DemC26_7
DemC26_8
DemC27_1
DemC27_2
DemC27_3
DemC27_4
DemC27_5
DemC31
DemC31_TEXT
DemC32
DemC33_R1
DemC37
DemC39
Q28
DemM11
DemM11_TEXT
DemM12
DemC51
RW1_1
RW1_2
RW1_3
RW1_4
RW1_5
RW1_5_TEXT
RW1_6
RW1_7
RW1_7_TEXT
RW1_8
RW2
RW3
RW4
RW5_1
Q317_2
RW5_2
RW5_4
RW5_5
RW5_6
RW5_7
RW5_8
RW5_9
RW5_11
RW5_13
RW5_14
RW5_15
RW5_16
RW5_16_TEXT
RW5_17
RW5_17_TEXT
RW6
RW7_1_TEXT
RW7_2_TEXT
RW7_3_TEXT
RW7.2_1_TEXT
RW7.2_2_TEXT
RW8
RW9
RW10
RW11
RW12
Q48
RW15
RW16_1
RW16_2
RW16_3
RW16_4
RW17
RW18_2
RW18_3
RW18_4
RW18_5
RW18_5_TEXT
RW18_6
RW19_1
RW19_2
RW19_3
RW19_3_TEXT
RW20
RW21_1_1
RW21_1_3
RW21_2_1


### Starting from week5

In [63]:
def processed_v2(data_path, num_path, aligned, lookup, week):
    data = pd.read_csv(data_path, encoding = "ISO-8859-1", dtype=str, keep_default_na=False, na_values='')
    num = pd.read_csv(num_path, encoding = "ISO-8859-1", dtype=str, keep_default_na=False, na_values='')
    print('processing RW27..')
    data = handle_RW27(num, data, week)
    print('processing text exchange')
    processed = ext_txt(data, week, aligned, lookup, debug=True, debug_col='DISG2.2_23')
    return processed

In [64]:
# week 5
week5_data = processed_v2(data_files['Week5'], num_files['Week5'], aligned, lookup, 'Week5')
w5_randomizer = pd.read_csv(randomizer_files['Week5'], encoding = "ISO-8859-1", dtype=str, keep_default_na=False, na_values=['', 'nan'])
w5_randomizer.columns = clean_column_names(w5_randomizer.columns)
w5_randomizer.drop(['V3', 'V4', 'V5', 'PROLIFIC_PID'], inplace=True, axis=1)
w5_randomizer = unify_randomizer_names(w5_randomizer, 'Week5', aligned, lookup)


week5_data = week5_data.merge(w5_randomizer, how='left', on='V1')
week5_data = preproc.clean_text_columns(week5_data, aligned)
week5_data.to_csv('output/cleaned_data/tmp/Week5.csv', index=False)

# Week5 testing
# test.check_items(data_files['Week5'], 'output/cleaned_data/tmp/week5.csv')
# test.check_nums(num_files['Week5'], 'output/cleaned_data/tmp/week5.csv')

processing RW27..
Index(['V1', 'V2', 'V3', 'V4', 'V5', 'PROLIFIC_PID', 'CVDcomp', 'PENcomp',
       'CVD', 'PEN',
       ...
       'VSA1_5', 'VSA1_6', 'norm1', 'norm2', 'norm3', 'norm4', 'longitudinal2',
       'feedback', 'resource1', 'resource2'],
      dtype='object', length=955)
processing text exchange


V1
V2
V3
V4
V5
PROLIFIC_PID
CVDcomp
PENcomp
CVD
PEN
TASK1
TASK2
TASK3
TASK4
TASK5
pavLink
Mobile
PID Missing
browser_1_TEXT
browser_2_TEXT
browser_3_TEXT
browser_4_TEXT
browser_5_TEXT
browser_6_TEXT
browser_7_TEXT
Q4_1
Q4_2
Q4_3
Q4_4
Q1_1
Q1_2
Q1_3
Q1_4
consent
Declined
longitudinal1
commit
resource_pre
longitudinal2
NIHE1_9
DISG1.2_23
Fed_13
RW23
RP1
RP2
RP3
RP4
RP5
RP6
RP7
RP8
RP9_1
RP9_2
RP9_3
RP9_4
RP9_5
RP9_5_TEXT
RP10
DemW1
DemW2
DemW3
DemW4
DemW5
DemW6
DemW7
DemW8
DemW9_1
DemW9_2
DemW9_3
DemW9_4
DemW9_5
DemW9_6
DemW9_7
DemW9_8
DemW9_9
DemW9_10
DemW9_11
DemW9_11_TEXT
DemW10_R3
DemW11
DemW12
DemW13
DemW14
DemW15_1
DemW15_2
DemW15_3
DemW15_4
DemW15_5
DemW15_5_TEXT
DemW16
DemW17_1
DemW17_2
DemW17_3
DemW17_3_TEXT
DemW18_R2
DemW19_1
DemW19_2
DemW19_3
DemW19_4
DemW19_5
DemW19_5_TEXT
DemW19_6
DemW20_1
DemW20_2
DemW20_3
DemW20_3_TEXT
DemW20_4
DemW21_1_TEXT
DemW21_2_TEXT
DemW21_3_TEXT
RW1_1
RW1_2
RW1_3
RW1_4
RW1_5
RW1_5_TEXT
RW1_6
RW1_7
RW1_7_TEXT
RW1_8
RW2
RW3
RW4
RW5_1
RW5_2
RW5_3
RW5_4

In [65]:
# week 6
week6_data = processed_v2(data_files['Week6'], num_files['Week6'], aligned, lookup, 'Week6')
week6_data = week6_data.loc[~week6_data['PROLIFIC_PID'].isna()]
w6_randomizer = pd.read_csv(randomizer_files['Week6'], encoding = "ISO-8859-1", dtype=str, keep_default_na=False, na_values=['', 'nan'])
w6_randomizer.columns = clean_column_names(w6_randomizer.columns)
w6_randomizer.drop(['V3', 'V4', 'V5', 'PROLIFIC_PID'], inplace=True, axis=1)
w6_randomizer = unify_randomizer_names(w6_randomizer, 'Week6', aligned, lookup)

week6_data = week6_data.merge(w6_randomizer, how='left', on='V1')
week6_data = preproc.clean_text_columns(week6_data, aligned)
week6_data.to_csv('output/cleaned_data/tmp/Week6.csv', index=False)

# Week6 testing
# test.check_items(data_files['Week6'], 'output/cleaned_data/tmp/week6.csv')
# test.check_nums(num_files['Week6'], 'output/cleaned_data/tmp/week6.csv')

processing RW27..
Index(['V1', 'V2', 'V3', 'V4', 'V5', 'PROLIFIC_PID', 'CVDcomp', 'PENcomp',
       'CVD', 'PEN',
       ...
       'Theo4_TEXT', 'Theo5', 'Theo5_TEXT', 'Theo6', 'Theo7_1_TEXT',
       'Theo7_2_TEXT', 'longitudinal2', 'feedback', 'resource1', 'resource2'],
      dtype='object', length=1055)
processing text exchange


V1
V2
V3
V4
V5
PROLIFIC_PID
CVDcomp
PENcomp
CVD
PEN
TASK1
TASK2
TASK3
TASK4
TASK5
pavLink
Mobile
PID Missing
browser_1_TEXT
browser_2_TEXT
browser_3_TEXT
browser_4_TEXT
browser_5_TEXT
browser_6_TEXT
browser_7_TEXT
Q4_1
Q4_2
Q4_3
Q4_4
Q1_1
Q1_2
Q1_3
Q1_4
consent
Declined
longitudinal1
commit
resource_pre
longitudinal2
DISG2.2_23
ReSe1_23
Fed_13
RW23
RP3
RP7
RP9_1
RP9_2
RP9_3
RP9_4
RP9_5
RP9_5_TEXT
DemW1
DemW2
DemW3
DemW4
DemW5
DemW6
DemW7
DemW8
DemW9_1
DemW9_2
DemW9_3
DemW9_4
DemW9_5
DemW9_6
DemW9_7
DemW9_8
DemW9_9
DemW9_10
DemW9_11
DemW9_11_TEXT
DemW10_R3
DemW11
DemW12
DemW13
DemW14
DemW15_1
DemW15_2
DemW15_3
DemW15_4
DemW15_5
DemW15_5_TEXT
DemW16
DemW17_1
DemW17_2
DemW17_3
DemW17_3_TEXT
DemW18_R2
DemW19_1
DemW19_2
DemW19_3
DemW19_4
DemW19_5
DemW19_5_TEXT
DemW19_6
DemW20_1
DemW20_2
DemW20_3
DemW20_3_TEXT
DemW20_4
DemW21_1_TEXT
DemW21_2_TEXT
DemW21_3_TEXT
RW1_1
RW1_2
RW1_3
RW1_4
RW1_5
RW1_5_TEXT
RW1_6
RW1_7
RW1_7_TEXT
RW1_8
RW2
RW3
RW4
RW5_1
RW5_2
RW5_3
RW5_4
RW5_5
RW5_6
RW5_7
RW5_8
RW5

EmbT1
EmbT2
EmbT3
EmbC1
EmbC2
EmbC3
EmbS1
EmbS2
EmbS3
EmbB1
EmbB2
EmbB3
Emb5
Emb6
Emb7
CvPP1_1
CvPP1_2
CvPP1_3
CvPP1_4
CvPP1_5
CvPP1_6
CvPP1_7
CvPP1_8
CvPP1_9
CvPP1_10
CvPP1_11
CvPP1_12
feedback
DO-Q-City
DO-Q-DISG2.1
DO-Q-DISG2.2
DO-Q-EmbB2
DO-Q-EmbC2
DO-Q-EmbO2
DO-Q-EmbS2
DO-Q-EmbT2
DO-Q-EMSB1
DO-Q-EMSC1
DO-Q-Fear2
DO-Q-Fed
DO-Q-RW6_1
DO-Q-State
DO-Q-Thrm1


In [66]:
w6_randomizer['DO-Q-EmbB2']

0       Display Order: In relation to the book, where ...
1                                                     NaN
2                                                     2|1
3                                                     NaN
4                                                     NaN
                              ...                        
1196                                                  NaN
1197                                                  NaN
1198                                                  NaN
1199                                                  NaN
1200                                                  NaN
Name: DO-Q-EmbB2, Length: 1201, dtype: object

In [67]:
week6_data['DO-Q-EmbB2']

0       Display Order: In relation to the book, where ...
1                                                     1|2
2                                                     nan
3                                                     nan
4                                                     nan
                              ...                        
1174                                                  nan
1175                                                  nan
1176                                                  nan
1177                                                  nan
1178                                                  1|2
Name: DO-Q-EmbB2, Length: 1179, dtype: object

In [68]:
week6_data.shape

(1179, 1075)

In [69]:
# Wave 7
week7_data = processed_v2(data_files['Wave7'], num_files['Wave7'], aligned, lookup, 'Wave7')
w7_randomizer = pd.read_csv(randomizer_files['Wave7'], encoding = "ISO-8859-1", dtype=str, keep_default_na=False, na_values=['', 'nan'])
w7_randomizer.columns = clean_column_names(w7_randomizer.columns)
w7_randomizer.drop(['V3', 'V4', 'V5', 'PROLIFIC_PID'], inplace=True, axis=1)
w7_randomizer = unify_randomizer_names(w7_randomizer, 'Wave7', aligned, lookup)


week7_data = week7_data.merge(w7_randomizer, how='left', on='V1')
week7_data = preproc.clean_text_columns(week7_data, aligned)
week7_data.to_csv('output/cleaned_data/tmp/Wave7.csv', index=False, encoding='utf-8')

# Week6 testing
# test.check_items(data_files['Wave7'], 'output/cleaned_data/tmp/wave7.csv')
# test.check_nums(num_files['Wave7'], 'output/cleaned_data/tmp/wave7.csv')

processing RW27..
Index(['V1', 'V2', 'V3', 'V4', 'V5', 'PROLIFIC_PID', 'CVDcomp', 'PENcomp',
       'CVD', 'PEN',
       ...
       'VSA1_5', 'VSA1_6', 'norm1', 'norm2', 'norm3', 'norm4', 'longitudinal2',
       'feedback', 'resource1', 'resource2'],
      dtype='object', length=938)
processing text exchange
GFPS7_1
{'Numeric': {'happenedtome': 1.0, 'witnessedithappeningtosomeoneelse': 2.0, 'learnedaboutithappeningtoaclosefamilymemberorclosefriend': 3.0, 'noneofthese': 4.0, 'prefernottodisclose': 5.0, 'doesnotapply': 6.0}}
['During the protests in the past two weeks, did you or your family / close friends experience any...-Damage to home or personal property'
 'None of these'
 'Learned about it happening to a close family member or close friend'
 'Prefer not to disclose' 'Does not apply'
 'Witnessed it happening to someone else' 'Happened to me']


V1
V2
V3
V4
V5
PROLIFIC_PID
CVDcomp
PENcomp
CVD
PEN
TASK1
TASK2
TASK3
TASK4
TASK5
pavLink
Mobile
PID Missing
browser_1_TEXT
browser_2_TEXT
browser_3_TEXT
browser_4_TEXT
browser_5_TEXT
browser_6_TEXT
browser_7_TEXT
Q4_1
Q4_2
Q4_3
Q4_4
Q1_1
Q1_2
Q1_3
Q1_4
consent
Declined
longitudinal1
commit
resource_pre
longitudinal2
DISG2.2_23
RW23
GFPS2_11
RP3
RP7
RP9_1
RP9_2
RP9_3
RP9_4
RP9_5
RP9_5_TEXT
DemW1
DemW2
DemW3
DemW4
DemW5
DemW6
DemW7
DemW8
DemW9_1
DemW9_2
DemW9_3
DemW9_4
DemW9_5
DemW9_6
DemW9_7
DemW9_8
DemW9_9
DemW9_10
DemW9_11
DemW9_11_TEXT
DemW10_R4
DemW22
DemW11
DemW12
DemW13
DemW14
DemW15_1
DemW15_2
DemW15_3
DemW15_4
DemW15_5
DemW15_5_TEXT
DemW16
DemW17_1
DemW17_2
DemW17_3
DemW17_3_TEXT
DemW18_R2
DemW18.1
DemW18.2
DemW19_1
DemW19_2
DemW19_3
DemW19_4
DemW19_5
DemW19_5_TEXT
DemW19_6
DemW20_1
DemW20_2
DemW20_3
DemW20_3_TEXT
DemW20_4
DemW21_1_TEXT
DemW21_2_TEXT
DemW21_3_TEXT
DemM1_1
DemM1_2
DemM1_3
DemM1_4
DemM1_5
DemM1_6
DemM1_7
DemM2
DemM3_R2
DemM4
DemM5
DemM6
DemM6_TEXT
DemM7
DemM8_1
D

In [70]:
week7_data['DO_Event_Surveys']

0            Display Order: Block Randomizer FL_139437108
1       FL_139443357|FL_12|FL_139444442|FL_139443358|F...
2       FL_12|FL_139443357|FL_139443358|FL_139444442|F...
3       FL_139444442|FL_139384328|FL_139443358|FL_12|F...
4       FL_139443357|FL_139444442|FL_139443358|FL_12|F...
                              ...                        
1161    FL_12|FL_139444442|FL_139443358|FL_139384328|F...
1162    FL_139443357|FL_139443358|FL_139384328|FL_12|F...
1163    FL_139443357|FL_12|FL_139443358|FL_139384328|F...
1164    FL_12|FL_139443358|FL_139443357|FL_139384328|F...
1165    FL_139443358|FL_139444442|FL_139384328|FL_1394...
Name: DO_Event_Surveys, Length: 1166, dtype: object

In [71]:
# Wave 8
week8_data = processed_v2(data_files['Wave8'], num_files['Wave8'], aligned, lookup, 'Wave8')
w8_randomizer = pd.read_csv(randomizer_files['Wave8'], encoding = "ISO-8859-1", dtype=str, keep_default_na=False, na_values=['', 'nan'])
w8_randomizer.columns = clean_column_names(w8_randomizer.columns)
w8_randomizer.drop(['V3', 'V4', 'V5', 'PROLIFIC_PID'], inplace=True, axis=1)
w8_randomizer = unify_randomizer_names(w8_randomizer, 'Wave8', aligned, lookup)


week8_data = week8_data.merge(w8_randomizer, how='left', on='V1')
week8_data = preproc.clean_text_columns(week8_data, aligned)
week8_data.to_csv('output/cleaned_data/tmp/Wave8.csv', index=False, encoding='utf-8')

processing RW27..
Index(['V1', 'V2', 'V3', 'V4', 'V5', 'Unnamed: 5', 'PROLIFIC_PID', 'CVDcomp',
       'PENcomp', 'CVD',
       ...
       'DO-Q-EMSC1', 'DO-Q-CEBL6', 'DO-Q-CEBL5', 'DO-Q-CEBL2', 'DO-Q-CEBL3',
       'DO-Q-CEBL7', 'DO-Q-CEBL3.1', 'DO-Q-CEBL1', 'DO-Q-Fear2', 'DO-Q-Thrm1'],
      dtype='object', length=1222)
processing text exchange


V1
V2
V3
V4
V5
PROLIFIC_PID
CVDcomp
PENcomp
CVD
PEN
TASK1
TASK2
TASK3
TASK4
TASK5
pavLink
Mobile
PID Missing
browser_1_TEXT
browser_2_TEXT
browser_3_TEXT
browser_4_TEXT
browser_5_TEXT
browser_6_TEXT
browser_7_TEXT
Q4_1
Q4_2
Q4_3
Q4_4
Q1_1
Q1_2
Q1_3
Q1_4
consent
Declined
longitudinal1
commit
resource_pre
longitudinal2
NIHE1_9
DISG1.2_23
ReSe1_23
RW23
RP3
RP7
RP9_1
RP9_2
RP9_3
RP9_4
RP9_5
RP9_5_TEXT
DemC2
DemW1
DemW2
DemW3
DemW4
DemW5
DemW6
DemW7
DemW8
DemW9_1
DemW9_2
DemW9_3
DemW9_4
DemW9_5
DemW9_6
DemW9_7
DemW9_8
DemW9_9
DemW9_10
DemW9_11
DemW9_11_TEXT
DemW10_R4
DemW22
DemW11
DemW12
DemW13
DemW14
DemW15_1
DemW15_2
DemW15_3
DemW15_4
DemW15_5
DemW15_5_TEXT
DemW16
DemW17_1
DemW17_2
DemW17_3
DemW17_3_TEXT
DemW18_R2
DemW18.1
DemW18.2
DemW19_1
DemW19_2
DemW19_3
DemW19_4
DemW19_5
DemW19_5_TEXT
DemW19_6
DemW20_1
DemW20_2
DemW20_3
DemW20_3_TEXT
DemW20_4
DemW21_1_TEXT
DemW21_2_TEXT
DemW21_3_TEXT
RW1_1
RW1_2
RW1_3
RW1_4
RW1_5
RW1_5_TEXT
RW1_6
RW1_7
RW1_7_TEXT
RW1_8
RW2
RW3
RW4
RW5_1
RW5_2
RW5_3
R

GFPS12v2_7
GFPS12v2_8
GFPS17v2_1
GFPS17v2_2
GFPS17v2_3
GFPS17v2_6
GFPS17v2_8
GFPS17v2_9
GFPS17v2_10
GFPS17v2_11
GFPS17v2_12
GFPS18v2
GFPS25v2_5
GFPS25v2_6
GFPS25v2_7
GFPS26v2
GFPS27v2
GFPS29v2
GFPS34v2_5
GFPS35v2_1
GFPS35v2_2
GFPS35v2_3
GFPS35v2_4
GFPS36v2_1
GFPS36v2_2
GFPS36v2_3
GFPS36v2_4
GFPS37v2_1
GFPS37v2_2
GFPS37v2_3
GFPS37v2_4
GFPS38v2_1
GFPS38v2_2
GFPS38v2_3
GFPS38v2_4
DO_Main_Surveys
DO-Q-CEBL1
DO-Q-CEBL2
DO-Q-CEBL3
DO-Q-cons2
DO-Q-DISG1.1
DO-Q-DISG1.2
DO-Q-EmbB2
DO-Q-EmbC2
DO-Q-EmbO2
DO-Q-EmbS2
DO-Q-EmbT2
DO-Q-EMSB1
DO-Q-EMSC1
DO-Q-Fear2
DO-Q-GFPS11v2
DO-Q-GFPS12v2
DO-Q-GFPS17v2
DO-Q-GFPS2v2
DO-Q-GFPS4v2
DO-Q-GFPS5v2
DO-Q-GFPS6v2
DO-Q-GFPS7v2
DO-Q-RW6_1
DO-Q-Thrm1


### Combining the data


In [72]:
weeks = ['Week1', 'Week2', 'Week3', 'Week4', 'Week5', 'Week6', 'Wave7', 'Wave8']

In [73]:
week_dats = []
wave = 1

for week in weeks:
    path = 'output/cleaned_data/tmp/'+ week + '.csv'
    week_dat = pd.read_csv(path, dtype=str, keep_default_na=False, na_values='')
    week_dat = week_dat.iloc[1:].reset_index(drop=True)
    
    week_dat.loc[:, 'wave'] = str(wave)
    wave = wave + 1
    
    week_dats.append(week_dat)

In [74]:
tmp = week_dats[0]

for i in range(1, len(week_dats)):
    tmp = tmp.append(week_dats[i], ignore_index=True, sort=False)
    

In [75]:
variables = aligned.loc[1:, ['unified_variable_names', 'Qualtrics-Text']]

In [76]:
rm_cols = ['2.1', 'Q364', 'Q239', 'Q3', 'Q2']


In [77]:
tmp_cleaned = tmp.drop(rm_cols, axis=1)

In [78]:
tmp_cleaned.columns

Index([' ', 'AnxS1_1', 'AnxS1_10', 'AnxS1_11', 'AnxS1_12', 'AnxS1_13',
       'AnxS1_14', 'AnxS1_15', 'AnxS1_16', 'AnxS1_17',
       ...
       'NEO', 'PC5.3', 'DO-Q-GFPS17v2', 'DO-Q-GFPS6v2', 'DO-Q-GFPS7v2',
       'DO-Q-GFPS2v2', 'DO-Q-GFPS12v2', 'DO-Q-GFPS5v2', 'DO-Q-GFPS4v2',
       'DO-Q-GFPS11v2'],
      dtype='object', length=2641)

In [79]:
tmp_cleaned.loc[tmp_cleaned['wave']=='4', ' ']

4780    NaN
4781    NaN
4782    NaN
4783    NaN
4784    NaN
       ... 
6169    NaN
6170    NaN
6171    NaN
6172    NaN
6173    NaN
Name:  , Length: 1394, dtype: object

In [80]:
tmp_cleaned.shape

(10899, 2641)

In [81]:
tmp_cleaned.loc[tmp_cleaned['wave']=='1'].shape

(1797, 2641)

In [82]:
tmp_cleaned.loc[tmp_cleaned['wave']=='2'].shape

(1521, 2641)

In [83]:
tmp_cleaned.loc[tmp_cleaned['wave']=='3'].shape

(1462, 2641)

In [84]:
tmp_cleaned.loc[tmp_cleaned['wave']=='4'].shape

(1394, 2641)

In [85]:
tmp_cleaned.loc[tmp_cleaned['wave']=='5'].shape

(1282, 2641)

In [86]:
tmp_cleaned.loc[tmp_cleaned['wave']=='6'].shape

(1178, 2641)

In [87]:
tmp_cleaned.loc[tmp_cleaned['wave']=='7'].shape

(1165, 2641)

In [88]:
tmp_cleaned.loc[tmp_cleaned['wave']=='8'].shape

(1100, 2641)

In [89]:
tmp_cleaned.to_csv('output/Wave1-8.csv', index=False)

In [90]:
tmp_cleaned.loc[tmp_cleaned['PROLIFIC_PID']=='5cbe7fdb82218d00172aa4f3', 'NEO_N_z-score']

59       0.12109375
2337            NaN
4060            NaN
5713            NaN
6697            NaN
7929            NaN
9115            NaN
10646    1.29296875
Name: NEO_N_z-score, dtype: object

In [91]:
tmp_cleaned.loc[tmp_cleaned['wave']=='1', 'SNI_Ex_Network_Diversity_Orig_R_2019']

0       1
1       2
2       3
3       3
4       2
       ..
1792    1
1793    7
1794    6
1795    6
1796    4
Name: SNI_Ex_Network_Diversity_Orig_R_2019, Length: 1797, dtype: object

In [92]:
tmp_cleaned.loc[tmp_cleaned['wave']=='2', 'DO-Q-Fear2']

1797    3|1|2|4|7|5|6
1798    3|2|6|5|7|4|1
1799    4|5|1|2|7|3|6
1800    3|6|5|4|7|2|1
1801    4|2|1|6|3|7|5
            ...      
3313    3|4|1|2|5|6|7
3314    2|5|4|7|1|6|3
3315    3|6|7|5|4|1|2
3316    7|3|5|2|1|6|4
3317    1|2|3|6|4|5|7
Name: DO-Q-Fear2, Length: 1521, dtype: object

In [93]:
tmp_cleaned.loc[tmp_cleaned['wave']=='2','LEC5_FC_total_raw']

1797    0
1798    0
1799    0
1800    0
1801    7
       ..
3313    3
3314    0
3315    5
3316    2
3317    2
Name: LEC5_FC_total_raw, Length: 1521, dtype: object

In [94]:
tmp_cleaned

,,AnxS1_1,AnxS1_10,AnxS1_11,AnxS1_12,AnxS1_13,AnxS1_14,AnxS1_15,AnxS1_16,AnxS1_17,...,NEO,PC5.3,DO-Q-GFPS17v2,DO-Q-GFPS6v2,DO-Q-GFPS7v2,DO-Q-GFPS2v2,DO-Q-GFPS12v2,DO-Q-GFPS5v2,DO-Q-GFPS4v2,DO-Q-GFPS11v2
0,32,3.0,3.0,2.0,2.0,3.0,1.0,2.0,3.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,55,1.0,1.0,2.0,3.0,3.0,1.0,1.0,1.0,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,64,3.0,3.0,3.0,1.0,1.0,1.0,2.0,2.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,23,2.0,2.0,1.0,2.0,2.0,2.0,1.0,2.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,25,2.0,1.0,2.0,2.0,2.0,3.0,2.0,2.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10894,NaN,2.0,2.0,2.0,2.0,2.0,2.0,2.0,1.0,2.0,...,1,1,9|3|1|11|10|12|8|6|2,12|7|9|11|4|10,2|3|1|4,1|4|5|2|9|8|3|10|7,7|6|5|4|8|3|1|2,4|5|2|3|1|7,2|7|4|1|6,1|2|7|6|5|4|3
10895,NaN,2.0,2.0,1.0,1.0,1.0,4.0,2.0,1.0,2.0,...,1,1,10|8|12|3|1|11|6|9|2,7|4|10|12|9|11,2|3|1|4,5|7|10|8|9|2|1|4|3,1|6|2|8|4|7|3|5,2|1|4|3|5|7,2|6|1|4|7,6|7|1|4|2|5|3
10896,NaN,2.0,2.0,2.0,3.0,1.0,1.0,2.0,2.0,3.0,...,1,1,1|8|10|12|9|11|3|2|6,11|4|12|9|10|7,2|3|1|4,7|4|5|8|10|2|1|3|9,7|3|8|4|2|5|6|1,2|4|1|5|3|7,4|1|7|6|2,4|6|5|1|3|2|7
10897,NaN,1.0,1.0,1.0,4.0,4.0,3.0,1.0,1.0,3.0,...,1,1,9|2|6|10|8|12|3|11|1,11|12|10|4|9|7,1|2|3|4,1|8|10|3|2|5|9|4|7,3|1|8|7|6|5|4|2,5|3|2|1|4|7,1|7|4|6|2,2|7|1|6|5|4|3


# Combining the different datasets, validation, raking and polific_samples into one

In [43]:
dat = pd.read_csv('output/Wave1-8_A-E.csv', encoding = "ISO-8859-1", dtype=str, keep_default_na=False, na_values=['', 'nan'])

In [44]:
validation_w1_8 = pd.read_csv('Working_Data/other_info/data/validation_passCriterion_perSub_perWave_w1-8_prlfc.csv')
validation_wA_E = pd.read_csv('Working_Data/other_info/data/validation_passCriterion_perSub_perWave_wA-E_conte.csv')
rake = pd.read_csv('Working_Data/other_info/data/raked_data.csv')

In [45]:
dat.loc[dat['PROLIFIC_PID']=='5cbe7fdb82218d00172aa4f3', 'NEO_N_z-score']

59       0.12109375
3212            NaN
4037            NaN
5690            NaN
6674            NaN
7906            NaN
9092            NaN
10239    1.29296875
Name: NEO_N_z-score, dtype: object

## (Outdated, do not run in this file)Clean up PCL_1_1 - PCL_20_5



In [73]:
import pandas as pd
import numpy as np
from natsort import natsorted

def construct_vars(prefix, n, postfix=None, skip=[]):
    ls = []
    for i in range(n):
        if (i+1) not in skip:
            out = prefix + str(i+1)
            if postfix:
                out = out + postfix
            ls.append(out)
    return ls


def extract_values(row):
    out = np.nan
    for i in range(len(row)):
        if row[i] is not np.nan:
            out = str(i+1)
    return out




In [74]:
data = tmp_cleaned.reindex(natsorted(tmp_cleaned.columns), axis=1)
w5_PCL = data.loc[data['wave']=='5', 'PCL1_1_1':'PCL1_20_5']

var_names = construct_vars('PCL1_', 20)

In [75]:
PCL_wrong_vars = []
for var in var_names:
    tmp = construct_vars(var+'_', 5)
    values = data[tmp].apply(extract_values, axis=1).values
    data[var] = values
    PCL_wrong_vars.append(tmp)

PCL_wrong_vars = np.ravel(PCL_wrong_vars)
w5_PCL = data.loc[data['wave']=='5', var_names]

data = data.drop(columns=PCL_wrong_vars)



In [76]:
## Clean up ids and fill in matching demographics
# fill in NAs for demographic
def map_w1_to_other(col_name, dat):
    dictionary = dat.loc[dat['wave']=='1', ['PROLIFIC_PID', col_name]].set_index('PROLIFIC_PID').to_dict()
    dat.loc[dat[col_name].isna(), col_name] = dat.loc[dat[col_name].isna(), 'PROLIFIC_PID'].map(dictionary[col_name])
    return dat

In [77]:
data['PROLIFIC_PID'] = data['PROLIFIC_PID'].str.replace(' ', '')
data = map_w1_to_other('DemC9', data)
data = map_w1_to_other('DemW3', data)
data = map_w1_to_other('DemC8', data)

data = data.loc[~data['PROLIFIC_PID'].isna()]
data.to_csv('output/Wave1-7.csv', index=False)

### (Outdated) Handling Conte data

In [52]:
Conte = {'WeekA': 'Working_Data/Week2_Conte/W2C_Qualtrics/COVID19_first_Conte_WEEK2_text.csv',
         'WeekAnT': 'Working_Data/Week2_Conte_noTask/WACnT_Qualtrics/COVID19_first_Conte_WEEK2__NoTasks_text.csv'}
Conte_nums = {"WeekA": 'Working_Data/WeekA_Conte/WAC_Qualtrics/COVID19_first_CONTE_WEEK2_num.csv',
              "WeekAnT": 'Working_Data/WeekA_Conte_noTask/WACnT_Qualtrics/COVID19_first_Conte_WEEK2__NoTasks_text.csv'}

In [57]:
raw1 = pd.read_csv(Conte['WeekA'], encoding = "ISO-8859-1", dtype=str, keep_default_na=False, na_values='')
raw2 = pd.read_csv(Conte['WeekAnT'], encoding = "ISO-8859-1", dtype=str, keep_default_na=False, na_values='')

In [58]:
raw = pd.concat([raw1, raw2.iloc[1:]]).reset_index(drop=True)

In [59]:
raw.shape

(48, 1336)

In [53]:
num1 = pd.read_csv(Conte_nums['WeekA'], encoding = "ISO-8859-1", dtype=str, keep_default_na=False, na_values='')
num2 = pd.read_csv(Conte_nums['WeekAnT'], encoding = "ISO-8859-1", dtype=str, keep_default_na=False, na_values='')

In [54]:
num = pd.concat([num1, num2.iloc[1:]]).reset_index(drop=True)

In [60]:
num.shape

(48, 1336)

In [61]:
data = preproc.handle_RW27(num, raw, week='Week2C')

In [63]:
processed = preproc.ext_txt(data, 'Week2C', aligned, lookup)

In [64]:
processed.to_csv('output/cleaned_data/Week2C.csv', index=False)

In [ ]:
raw.to_csv('Working_dat')

In [ ]:
num.to_csv()